# TWFS DuckDB EDA

這份 notebook 用來做最基本的探索式資料分析（EDA）：
- 檢查資料範圍與筆數
- 查指定銀行（關鍵字搜尋）
- 看單一銀行的逐月變化

> 先確認已執行：`python pipelines/build_databases.py --base-path .`


In [ ]:
from pathlib import Path
import duckdb

DB_PATH = Path("database/twfs.duckdb")
if not DB_PATH.exists():
    raise FileNotFoundError(f"DuckDB not found: {DB_PATH}. Please run build_databases.py first.")

con = duckdb.connect(str(DB_PATH))
print("Connected:", DB_PATH)


In [ ]:
summary_sql = """
SELECT
  COUNT(*) AS row_count,
  MIN(month_key) AS min_month,
  MAX(month_key) AS max_month,
  COUNT(DISTINCT dataset) AS datasets,
  COUNT(DISTINCT institution) AS institutions
FROM facts
"""
con.sql(summary_sql).df()


## 指定銀行搜尋

把 `BANK_KEYWORD` 改成你要找的銀行名稱，例如：
- `台灣銀行`
- `中國信託`
- `玉山`


In [ ]:
BANK_KEYWORD = "台灣銀行"

bank_sql = """
SELECT DISTINCT institution
FROM facts
WHERE institution LIKE '%' || ? || '%'
ORDER BY institution
"""

banks = con.execute(bank_sql, [BANK_KEYWORD]).df()
banks


In [ ]:
# 如果上一步有多個機構，這裡可改成其中一個完整名稱
TARGET_BANK = BANK_KEYWORD

sample_sql = """
SELECT month_key, dataset, item_zh, item_en, value_num
FROM facts
WHERE institution LIKE '%' || ? || '%'
  AND value_num IS NOT NULL
ORDER BY month_key DESC, dataset, item_en
LIMIT 200
"""

con.execute(sample_sql, [TARGET_BANK]).df()


In [ ]:
# 觀察指定銀行逐月總量（示意）
trend_sql = """
SELECT month_key, ROUND(SUM(value_num), 2) AS total_value
FROM facts
WHERE institution LIKE '%' || ? || '%'
  AND value_num IS NOT NULL
GROUP BY month_key
ORDER BY month_key
"""

trend_df = con.execute(trend_sql, [TARGET_BANK]).df()
trend_df.tail(24)


In [ ]:
# 可選：簡單視覺化（若環境有 matplotlib）
try:
    import matplotlib.pyplot as plt

    if not trend_df.empty:
        plt.figure(figsize=(12, 4))
        plt.plot(trend_df["month_key"], trend_df["total_value"])
        plt.title(f"{TARGET_BANK} monthly total value")
        plt.xticks(rotation=90)
        plt.tight_layout()
        plt.show()
except ModuleNotFoundError:
    print("matplotlib not installed. Skip plotting.")


## 延伸分析

你可以直接改 SQL 做更多 EDA：
- 不同 `dataset` 的最新月份比較
- `item_en` 指標排序
- 指定月份的機構排名
